# 자동 생성 도구 사용 예


## 필요한 패키지 가져오기


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## 클라이언트 생성하기

이 샘플에서는 LLM에 접근하기 위해 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models)를 사용할 것입니다.

`model`은 `gpt-4o-mini`로 정의되어 있습니다. GitHub Models 마켓플레이스에서 제공되는 다른 모델로 변경하여 다양한 결과를 확인해 보세요.

간단한 테스트로, `프랑스의 수도는 무엇인가요?`라는 간단한 프롬프트를 실행해 보겠습니다.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 함수 정의하기

이 예제에서는 에이전트가 사용할 수 있는 도구로, 이용 가능한 휴가지 목록과 그 이용 가능 여부를 제공하는 함수를 정의해 보겠습니다.

이 상황은 예를 들어 여행사가 여행 데이터베이스에 접근할 수 있는 시나리오라고 생각할 수 있습니다.

이 샘플을 진행하면서 에이전트가 호출할 수 있는 새로운 함수와 도구를 자유롭게 정의해 보세요.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## 함수 도구 정의하기  
에이전트가 `vacation_destinations`를 `FunctionTool`로 사용하도록 하려면, 이를 하나의 도구로 정의해야 합니다.  

또한, 사용자 요청과 관련된 작업에서 해당 도구가 어떤 용도로 사용되는지 에이전트가 이해할 수 있도록 도구에 대한 설명도 제공할 것입니다.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## 에이전트 정의하기

이제 아래 코드에서 에이전트를 생성할 수 있습니다. `system_message`를 정의하여 사용자들이 휴가지 목적지를 찾는 데 도움을 줄 수 있도록 에이전트에게 지침을 제공합니다.

또한 `reflect_on_tool_use` 매개변수를 true로 설정합니다. 이를 통해 도구 호출의 응답을 LLM이 받아 자연어로 응답을 보낼 수 있게 됩니다.

매개변수를 false로 설정하여 차이를 확인할 수도 있습니다.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## 에이전트 실행하기

이제 사용자가 도쿄 여행을 요청하는 초기 메시지와 함께 에이전트를 실행할 수 있습니다.

도시 목적지를 변경하여 해당 도시의 이용 가능성에 대해 에이전트가 어떻게 반응하는지 확인할 수 있습니다.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
